# Notebook to test and demo the reuseable functions in the codebase
added tested comments

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
from importlib import reload
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.metrics import r2_score

In [ ]:
from DRP_utils import data_preprocessing as dp_nb
reload(dp_nb)
from DRP_utils import model_selection as ms_nb
reload(ms_nb)
from DRP_utils import testing as t_nb

# Data reading and preprocessing

Uses and tests functions from my data_preprocessing module 

In [ ]:
#read and format target values

df_ic50 = pd.read_csv('downloaded_data_small/GDSC1_ic50.csv')
frame = {}
for d in np.unique(df_ic50['CELL_LINE_NAME']):
    cellDf = df_ic50[df_ic50['CELL_LINE_NAME'] == d]
    cellDf.index = cellDf['DRUG_NAME']
    frame[d] = cellDf['LN_IC50']
    
    
def remove_repeats_mean_gdsc1(frame, df_ic50): 
    new_frame = {}
    for cell_line in np.unique(df_ic50['CELL_LINE_NAME']):
        temp_subset = frame[cell_line].groupby(frame[cell_line].index).mean()
        new_frame[cell_line] = temp_subset
    return new_frame  

new_frame = remove_repeats_mean_gdsc1(frame, df_ic50)
ic50_df1 = pd.DataFrame(new_frame).T

_all_drugs = ic50_df1.columns

In [ ]:
#read and format phos data

phos_raw = pd.read_csv('downloaded_data_small/suppData2ppIndexPhospo.csv')
#makes index features 
phos_raw.index = phos_raw['col.name']
phos_raw.drop(columns='col.name', inplace=True)
#formats cell lines in the same way as in target value df. 
phos_raw.columns = [c.replace('.', '-') for c in phos_raw.columns]
phos_raw = phos_raw.T

In [ ]:
phos_raw, ic50_df1 = dp_nb.keep_overlapping(phos_raw, ic50_df1)
_all_cells = phos_raw.index
phos_raw.shape, ic50_df1.shape

In [ ]:
#log transfrom
phospho_log = np.log2(phos_raw).replace(-np.inf, 0)
#norm by cell line standard scale 
scale = StandardScaler()
phospho_ls = pd.DataFrame(scale.fit_transform(phospho_log.T),
                       columns = phospho_log.index,
                       index = phospho_log.columns).T
#drug one hot encoding
frame = {}
for i,d in enumerate(_all_drugs):
    hot_vec = np.zeros(len(_all_drugs))
    hot_vec[i] = 1
    frame[d] = hot_vec
one_hot_drugs = pd.DataFrame(frame)

In [ ]:
x_all, x_drug, y_list = dp_nb.create_all_drugs(phospho_ls, one_hot_drugs, ic50_df1, _all_cells)
all_cls_drugs = x_all.index + '.' + x_drug.index 
x_all.shape, x_drug.shape, len(y_list)

In [ ]:
#test train split
train_inds, test_inds = train_test_split(range(len(y_list)),test_size=0.8,
                                         random_state=42)
x_train, x_test = x_all.iloc[train_inds], x_all.iloc[test_inds]
y_train, y_test = y_list[train_inds], y_list[test_inds]
xdrug_train, xdrug_test = x_drug.iloc[train_inds], x_drug.iloc[test_inds]

In [ ]:
print(x_train.shape, xdrug_train.shape, len(y_train))
x_test.shape, xdrug_test.shape, len(y_test)

In [ ]:
#find the tranning cell lines, (needed for mean moel)
train_cls = [cl_drug.split('.')[0] for cl_drug in all_cls_drugs[train_inds]]
train_cls = np.unique(train_cls)
test_cls = [cl_drug.split('.')[0] for cl_drug in all_cls_drugs[test_inds]]
test_cls = np.unique(test_cls)
test_drugs = [cl_drug.split('.')[1] for cl_drug in all_cls_drugs[test_inds]]

# Model bulding

In [ ]:
def build_test_dnn(hps):
    num_uni, *_ = hps
    phos_input = layers.Input(shape=x_train.shape[1])
    x = layers.Dense(num_uni, activation='relu')(phos_input)
    drug_input = layers.Input(shape = (x_drug.shape[1]))
    concatenated = layers.concatenate([x, drug_input])
    hidd = layers.Dense(num_uni // 2, activation='relu')(concatenated)
    output_tensor = layers.Dense(1)(hidd)
    model = tf.keras.models.Model([phos_input,drug_input], output_tensor)
    model.compile(
        optimizer='rmsprop',
        loss=tf.keras.metrics.mean_squared_error,
        metrics=['mae'])
    return model

In [ ]:
build_test_dnn([64]).summary()

## Create a mean model for benchmarking
(model that just predicts the average truth value for each drug

In [ ]:
def create_mean_model(y):
    ''' creates mean model for y, tranning data 
    
    '''
    mm = {}
    for d in y.columns:
        mm[d] = np.mean(y[d].dropna())
    return mm


In [ ]:
mean_model = create_mean_model(ic50_df1.loc[train_cls])

## Create one hot encoded model for benchmarking 

In [ ]:
all_cls = phospho_ls.index
one_hot_cls = []
for i, cl in enumerate(all_cls):
    hot_cl = np.zeros(len(all_cls))
    hot_cl[i] = 1
    one_hot_cls.append(hot_cl)
one_hot_cls = pd.DataFrame(one_hot_cls)   
one_hot_cls.index = all_cls

In [ ]:
x_hot, x_drug_hot, y_hot = dp_nb.create_all_drugs(one_hot_cls, one_hot_drugs, ic50_df1, _all_cells)
x_hot.shape, x_drug_hot.shape, len(y_hot)

In [ ]:
x_train_hot, x_test_hot = x_hot.iloc[train_inds], x_hot.iloc[test_inds]
y_train_hot, y_test_hot = y_hot[train_inds], y_hot[test_inds]
xdrug_train_hot, xdrug_test_hot = x_drug_hot.iloc[train_inds], x_drug_hot.iloc[test_inds]
x_train_hot.shape, xdrug_train_hot.shape, len(y_train_hot)

In [ ]:
def build_dnn_hot(hps):
    num_uni, *_ = hps
    phos_input = layers.Input(shape=x_train_hot.shape[1])
    x = layers.Dense(num_uni, activation='relu')(phos_input)
    drug_input = layers.Input(shape = (x_drug.shape[1]))
    concatenated = layers.concatenate([x, drug_input])
    hidd = layers.Dense(num_uni // 2, activation='relu')(concatenated)
    output_tensor = layers.Dense(1)(hidd)
    model = tf.keras.models.Model([phos_input,drug_input], output_tensor)
    model.compile(
        optimizer='rmsprop',
        loss=tf.keras.metrics.mean_squared_error,
        metrics=['mae'])
    return model

In [ ]:
epochs = 100
loss, val_loss, *_ = ms_nb.run_cv(build_dnn_hot, 
                                  [x_train_hot, xdrug_train_hot],
                                  y_train_hot,
                                  [64],
                                  batch_size=4,
                                  k=3,
                                 epochs=epochs)

In [ ]:
ms_nb.plot_cv(loss, val_loss, epochs=epochs)

In [ ]:
2.01 - 0.068

In [ ]:
hot_benchmark = build_dnn_hot([54])
hot_benchmark.fit([x_train_hot, xdrug_train_hot], y_train_hot, epochs=56, batch_size=4)

# Model seleciton
Uses and tests functions from my model_selection module 

In [ ]:
epochs = 75
loss, val_loss, *_ = ms_nb.run_cv(build_test_dnn, 
                                  [x_train, xdrug_train],
                                  y_train,
                                  [64],
                                  batch_size=4,
                                  k=3,
                                 epochs=epochs)

In [ ]:
ms_nb.plot_cv(loss, val_loss, epochs=epochs)

## Hyper param opt
using run_random_hp_opt function

In [ ]:
def build_test_dnn(hps):
    '''Build neural network with keras
    
    Input hps as a dict
    '''
    num_hps = 1
    #check number of hyper prams in arg and needed matches
    assert len(hps) == num_hps 
    num_uni = hps['num_uni']
    
    phos_input = layers.Input(shape=x_train.shape[1])
    x = layers.Dense(num_uni, activation='relu')(phos_input)
    drug_input = layers.Input(shape = (x_drug.shape[1]))
    concatenated = layers.concatenate([x, drug_input])
    hidd = layers.Dense(num_uni // 2, activation='relu')(concatenated)
    output_tensor = layers.Dense(1)(hidd)
    model = tf.keras.models.Model([phos_input,drug_input], output_tensor)
    model.compile(
        optimizer='rmsprop',
        loss=tf.keras.metrics.mean_squared_error,
        metrics=['mae'])
    return model

In [ ]:
num_trails = 3
epochs = 10
hp_grid = {'num_uni': [32, 64, 128]}

opt_r, *_ = ms_nb.run_random_hp_opt(ParameterGrid(hp_grid),
                                    [x_train, xdrug_train],
                                    y_train,
                                   num_trails,
                                   build_test_dnn,
                                   epochs,
                                   batch_size=4)

In [ ]:
from collections import namedtuple
Results = namedtuple('tr',['rt', 'ft', 'xy' ])

r = Results(1,2,3)
r

In [ ]:
r

In [ ]:
opt_r

# Model testing
Uses and tests functions from my testing module 

In [ ]:
dnn_test = build_test_dnn([5])
dnn_test.fit([x_train, xdrug_train], y_train, epochs=5, batch_size=1)

In [ ]:
dnn_pre = dnn_test.predict([x_test, xdrug_test])
dnn_pre = dnn_pre.reshape(len(dnn_pre))
plt.scatter(dnn_pre, y_test)
r2_score(dnn_pre, y_test)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
pcm = ax.hist2d(dnn_pre.reshape(len(dnn_pre)), y_test, bins=75, cmap=plt.cm.jet)
fig.colorbar(pcm[3])
plt.show()
r2_score(dnn_pre, y_test)

In [ ]:
cl_results = t_nb.sort_results(dnn_pre, y_test, all_cls_drugs[test_inds], centered=0)
drug_results = t_nb.sort_results(dnn_pre, y_test, all_cls_drugs[test_inds], centered=1)

## Benchmark models testing

In [ ]:
#mean model 
fig, ax = plt.subplots(figsize=(8, 5))
pcm = ax.hist2d(mm_pre, y_test, bins=75, cmap=plt.cm.jet)
fig.colorbar(pcm[3])
plt.show()
r2_score(mm_pre, y_test)

In [ ]:
mm_pre = np.array([mean_model[d] for d in test_drugs])
cl_results_mm = t_nb.sort_results(mm_pre, y_test, all_cls_drugs[test_inds], centered=0)
drug_results_mm = t_nb.sort_results(mm_pre, y_test, all_cls_drugs[test_inds], centered=1)

In [ ]:
#one hot encoded model
hot_pre = hot_benchmark.predict([x_test_hot, xdrug_test_hot])
hot_pre = hot_pre.reshape(len(hot_pre))

fig, ax = plt.subplots(figsize=(8, 5))
pcm = ax.hist2d(hot_pre, y_test_hot, bins=75, cmap=plt.cm.jet)
fig.colorbar(pcm[3])
plt.show()
r2_score(hot_pre, y_test_hot)

## Compare mean model with real model

In [ ]:
def find_centric_statstic(result, statstic):
    '''for drug or cell line centric results finds a test statstic
    
    '''
    statstic_results = {}
    for key in result.keys():
        pre = result[key][0]
        true = result[key][1]
        statstic_results[key] = statstic(true, pre)
    return statstic_results

In [ ]:
cl_r2 = find_centric_statstic(cl_results, r2_score)
cl_r2_mm = find_centric_statstic(cl_results_mm, r2_score)

drug_r2 = find_centric_statstic(drug_results, r2_score)
drug_r2_mm = find_centric_statstic(drug_results_mm, r2_score)

In [ ]:
plt.hist(cl_r2.values())
plt.show()
plt.hist(cl_r2_mm.values())

In [ ]:
plt.hist(drug_r2.values())
plt.show()
plt.hist(drug_r2_mm.values())